In [1]:
#Imports
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.image as mpimg

#Remove parts of the image (the hood of the car and the part that's past the horizon) as these parts are useless
def removeImagePart(image, top, bottom):
  return image[top:bottom, :]

#Resize the image and reduce the time needed for training
def resizeImage(image, dimension):
  return cv2.resize(image, dimension, interpolation=cv2.INTER_AREA)

#Preprocessing the image by removing the unnecessary parts and resizing it to be 64 by 64
def preprocessImage(image, top=60, bottom=140, dimension=(64, 64)):
  return resizeImage(removeImagePart(image, top, bottom), dimension)

#Get certain view of the image with its angle randomly
def getImageAngle(row,randomCamera, angle=0.23):

    #Get left view. Thus, the angle is added
    if randomCamera == 0:
        image_path = row.left.split('\\')[-1]
        angle = row.angle + angle

    #Get center view. Thus, the angle is the same   
    elif randomCamera == 1:
        image_path = row.center.split('\\')[-1]
        angle = row.angle

    #Get right view. Thus, the angle is subtracted      
    else:
        image_path = row.right.split('\\')[-1]
        angle = row.angle - angle

    #Retieve the image
    image = mpimg.imread('../input/drivingcarimg/IMG/' + image_path)

    return image, angle

#Get a certain view of the image along with its angle
def randomCamera(row):
  randomCamera= np.random.randint(0, 3)
  image, angle = getImageAngle(row, randomCamera)
  return image, angle

#Flip the image half the time and change the angle accordingly
def flipImage(image, angle):
  if np.random.binomial(1, 0.5):
    return cv2.flip(image, 1), -angle
  else:
    return image, angle

#Change the brightness of the image
def adjustBrightness(image):
  #0.4(duller) and 1.5(brighter)
  gamma = np.random.uniform(0.4, 1.5)
  inverseGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** inverseGamma) * 255
                    for i in np.arange(0, 256)]).astype("uint8")
  return cv2.LUT(image, table)

#Shear the image horizontally to simulate a bending road by letting the pixels at the bottom of the image remain fixed 
#while the pixels at the top row are moved randomly to the left or right.
def shearImage(image, angle, shear_range=200):
  rows, cols, _ = image.shape
  dx = np.random.randint(-shear_range, shear_range + 1)
  randomPoint = [cols / 2 + dx, rows / 2]
  points1 = np.float32([[0, rows], [cols, rows], [cols / 2, rows / 2]])
  points2 = np.float32([[0, rows], [cols, rows], randomPoint])
  dangle = dx / (rows / 2) * 360 / (2 * np.pi * 25.0) / 6.0
  M = cv2.getAffineTransform(points1, points2)
  image = cv2.warpAffine(image, M, (cols, rows), borderMode=1)  
  return image, angle + dangle

#Shift the image vertically randomly to simulate the road in the second track.
def randomBumps(image, y_range=20):
  rows, cols, _ = image.shape
  dy = (y_range * np.random.uniform()) - (y_range / 2)
  M = np.float32([[1, 0, 0], [0, 1, dy]])
  return cv2.warpAffine(image, M, (cols, rows))

#Augment the images
def augmentData(image, angle):
  #90% of the images are sheared
  if np.random.binomial(1, 0.9):
    image, angle = shearImage(image, angle)
  image, angle = flipImage(image, angle)
  image = adjustBrightness(image)
  image = preprocessImage(image)
  image = randomBumps(image)
  return image, angle

#Give headers to the driving-log file and read it
def read_csv(path):
  headers = ['center', 'left', 'right', 'angle', 'throttle', 'brake', 'speed']
  return pd.read_csv(path, names=headers, skiprows=1)

#Get data needed for training(all the data obtained fom the driving-log file) and validation (all data with non-zero angles
#and only a 0.1 fraction of data with zero angles)
def getData():
    trainingData = read_csv('../input/drivingcar/driving_log.csv')   
    nonZeroTrainingData = trainingData[trainingData.angle != 0]
    zeroTrainingData = trainingData[trainingData.angle == 0]
    validationData = pd.concat([nonZeroTrainingData, zeroTrainingData.sample(frac=0.1)], ignore_index=True)
    return trainingData, validationData

#Get training batch
def trainingBatch(trainData, batchSize):
  total = len(trainData)
  while True:
    images = []
    angles = []
    #Get random batches
    randoms= np.random.randint(0, total, batchSize)
    for index in randoms:
      row = trainData.iloc[index]
      #Retieve the image along with its angle
      image, angle = randomCamera(row)
      ##Retieve the augmented image and its angle
      image, angle = augmentData(image, angle)
      images.append(image)
      angles.append(angle)

    yield np.array(images), np.array(angles)

#Get validation batch
def validationBatch(validationData, batchSize):
  total = len(validationData)
  current = 0
  while True:
    images = []
    angles = []
    for index in range(batchSize):
      row = validationData.iloc[current]
      #Retieve the center view of the image along with its angle
      image, angle = getImageAngle(row, 1)
      images.append(preprocessImage(image))
      angles.append(angle)
      current = (current + 1) % total

    yield np.array(images), np.array(angles)

In [2]:
#Imports   
from keras.layers import Dense, Flatten, Lambda, PReLU, MaxPooling2D, Dropout
from keras import regularizers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam


#Model architecture
def get_model():
  model = Sequential()
  #Normalization
  model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=(64, 64, 3)))

  #Convolutional and maxpooling layers
  model.add(Conv2D(filters=24, kernel_size=(5, 5), padding='valid', strides=(2, 2),kernel_regularizer=regularizers.l2(0)))
  model.add(PReLU())
  model.add(BatchNormalization())

  model.add(Conv2D(filters=24, kernel_size=(5, 5), padding='valid', strides=(2, 2),kernel_regularizer=regularizers.l2(0)))
  model.add(PReLU())
  model.add(BatchNormalization())

  model.add(Conv2D(filters=24, kernel_size=(5, 5), padding='valid', strides=(2, 2),kernel_regularizer=regularizers.l2(0)))
  model.add(PReLU())
  model.add(BatchNormalization())

  model.add(Conv2D(filters=64,kernel_size=( 3, 3), padding='same', strides=(1, 1),kernel_regularizer=regularizers.l2(0)))
  model.add(PReLU())
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

  model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1),kernel_regularizer=regularizers.l2(0)))
  model.add(PReLU())
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

  model.add(AveragePooling2D(pool_size=(3, 3), strides=(1, 1)))

  model.add(Flatten())

  # fully connected layer
  model.add(Dense(10))
  model.add(PReLU())
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(1, activation='tanh'))

  return model

def main():
  #Get training and validation data
  trainDriving, validDriving = getData()
  
  #Get model
  model = get_model()
  #Get the details of the architecture
  model.summary()

  #Generators for traindrivinging and validation
  BATCH = 100
  trainDrivingGenerator = trainingBatch(trainDriving, BATCH)
  validationDrivingGenerator = validationBatch(validDriving, BATCH)

  #Train
  EPOCHS = 30
  traindrivingS = 300 #22455/(5*3)
  VALIDS = 4491
  model.compile(optimizer=Adam( 1e-4), loss="mse")
  history = model.fit(trainDrivingGenerator,
                                steps_per_epoch=traindrivingS,
                                epochs=EPOCHS,
                                validation_data=validationDrivingGenerator,
                                validation_steps=VALIDS,
                                verbose=1)


  #Save the model
  from tensorflow.keras.models import save_model
  model.save('/kaggle/working/model2.h5')
  #model.save('./model.h5',save_format='h5')

  os.chdir(r'/kaggle/working')
  from IPython.display import FileLink
  FileLink(r'model2.h5')
    
    
if __name__ == '__main__':
  main()


2022-05-27 16:44:29.526812: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 24)        1824      
_________________________________________________________________
p_re_lu (PReLU)              (None, 30, 30, 24)        21600     
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 24)        96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 24)        14424     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 13, 13, 24)        4056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 24)        9

2022-05-27 16:44:30.710286: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
300/300 [==============================] - 1251s 4s/step - loss: 0.4179 - val_loss: 0.4473
Epoch 2/30
300/300 [==============================] - 1232s 4s/step - loss: 0.2372 - val_loss: 0.1255
Epoch 3/30
300/300 [==============================] - 1188s 4s/step - loss: 0.1674 - val_loss: 0.0864
Epoch 4/30
300/300 [==============================] - 1231s 4s/step - loss: 0.1377 - val_loss: 0.0745
Epoch 5/30
300/300 [==============================] - 1191s 4s/step - loss: 0.1200 - val_loss: 0.0740
Epoch 6/30
300/300 [==============================] - 1269s 4s/step - loss: 0.1068 - val_loss: 0.0675
Epoch 7/30
300/300 [==============================] - 1323s 4s/step - loss: 0.0979 - val_loss: 0.0643
Epoch 8/30
300/300 [==============================] - 1131s 4s/step - loss: 0.0906 - val_loss: 0.0635
Epoch 9/30
300/300 [==============================] - 1142s 4s/step - loss: 0.0853 - val_loss: 0.0602
Epoch 10/30
300/300 [==============================] - 1133s 4s/step - loss: 0.080